# 소프트맥스, MNIST Data Set

- 소프트맥스 함수
  - 분류에서 사용한다. 
  - 소프트맥스의 출력은 모든 입력 신호로부터 화살표를 받는다.
  - 소프트맥스의 함수의 분모에서 볼 수 있듯, 촐력층의 각 뉴런이 모든 입력 신호에서 영향을 받기 때문이다. 
  - 이상의 소프트맥스 함수를 구현해보자

In [1]:
import numpy as np

a = np.array([0.3,2.9,4])
exp_a = np.exp(a)
print(exp_a)

sum_exp_a = np.sum(exp_a)
print(sum_exp_a)

y = exp_a / sum_exp_a
print(y)

[ 1.34985881 18.17414537 54.59815003]
74.1221542101633
[0.01821127 0.24519181 0.73659691]


C:\Users\ehfus\Anaconda3\envs\dv2021\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ehfus\Anaconda3\envs\dv2021\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\ehfus\Anaconda3\envs\dv2021\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


- 위 논리 흐름을 파이썬 함수로 정의하자

In [2]:
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

- 소프트맥스 함수 구현시 주의할 점
  - 위에서 정의한 softmax() 함수의 코드는 컴퓨터로 계산할 때 결함이 있다.
    - 바로 오버플로 문제이다. 소프트맥스 함수는 지수 함수를 사용하는데, 지수함수란 것이 쉽게 아주 큰 값을 내지만 컴퓨터는 다룰 수 있는 범위가 한정되어 있어 수치가 커짐에 따라 결과 수치가 불안정해질 수 있다. 
    - 해결책 : 오버플로를 막기 위해 입력 신호 중 최댓값을 이용하여 지수 함수에서 빼준다.
    - 구현해보자

In [3]:
import warnings
warnings.filterwarnings('ignore')

a = np.array([1010,1000,990])
print(np.exp(a) / np.sum(np.exp(a))) 

c = np.max(a)

np.exp(a-c) / np.sum(np.exp(a-c))

[nan nan nan]


array([9.99954600e-01, 4.53978686e-05, 2.06106005e-09])

- 위에서 볼 수 있듯, 아무런 조치 없이 그냥 계산하면 nan이 반환된다.
- 하지만 입력 신호 중 최댓값을 빼주면 올바르게 계싼할 수 있다.
- 이를 바탕으로 소프트맥스 함수를 다시 구현해보자

In [4]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c) # 이게 오버플로 대책이다.
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

- 소프트맥스 함수의 특징

In [5]:
a = np.array([0.3,2.9,4])
y = softmax(a)
print(y)
print(np.sum(y))

[0.01821127 0.24519181 0.73659691]
1.0


- 즉, 소프트맥스 함수의 출력은 0에서 1사이의 실수이다. (그럴 수 밖에 없는 것이 소프트맥스 함수를 살펴보면 1을 넘을 수 없다)
- 출력의 총합은 1이며 소프트맥스 함수의 중요한 성질이 된다.
  - 이 성질 덕분에 소프트맥스 함수의 출력을 확률로 정의할 수 있다. 
  - 즉, 소프트맥스 함수를 이용함으로써 문제를 확률적(통계적)으로 대응할 수 있게 되는것이다.

---

- 출력층의 뉴런 수 정하기
  - 출력층의 뉴런 수는 풀려는 문제에 맞게 적절히 정해야 한다. 
  - 분류에서는 분류하고 싶은 클래스 수로 설정하는 것이 일반적이다.
     - 예를 들어 입력 이미지를 숫자 0부터 9 중 하나로 분류하는 문제라면 출력층의 뉴런을 10개로 설정한다. 

- 손글씨 숫자 인식
  - 이번 절에서는 이미 학습된 매개변수(가중치, 편향(임계값))를 사용하여 학습 과정은 생략하고 추론 과정만 구현한다.
  - 이 추론 과정을 신경망의 순전파(forward propagation)라고도 한다. 
    - 머신러닝과 마찬가지로 신경망도 두 단계를 거쳐 문제를 해결한다. 먼저 훈련 데이터 즉, 학습 데이터를 사용해 가중치 매개변수를 학습하고 추론 단계에서는 앞서 학습한 매개변수를 사용하여 입력 데이터를 분류한다.

- MNIST data set
  - 손글씨 숫자 이미지 집합
  - MNIST의 이미지 데이터는 28 $\times$ 28 크기의 회색조 이미지이며 각 픽셀은 0에서 255까지의 값을 취한다. 각 이미지에는 또한 7,2,1과 같이 그 이미지가 실제 의미하는 숫자가 레이블로 붙어 있다.

In [9]:
import sys,os
sys.path.append(os.pardir) # 부모 디렉토리의 파일을 가져올 수 있도록 설정
from dataset.mnist import load_mnist

(x_train, t_train),(x_test, t_test) = \
load_mnist(flatten = True, normalize = False)

# 각 데이터 형상 출력해보자
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


- 첫 번째 인수인 normalize는 입력 이미지의 픽셀값을 0~1 사이의 값으로 정규화할지를 정한다. 
- flatten은 입력 이미지를 평탄하게, 즉 1차원 배열로 만들지를 정한다. 
  - False로 설정하면 입력 이미지를 1 x 28 x 28의 3차원 배열로, True로 설정하면 784개의 원소로 이루어진 1차원 배열로 저장한다. 

---
- 참고) 파이썬에는 pickle이라는 편리한 기능이 있다. 이는 프로그램 실행 중에 특정 객체를 파일로 저장하는 기능이다. 저장해준 pickle 파일을 로드하면 실행 당시의 객체를 즉시 복원할 수 있다. MNIST 데이터셋을 읽는 load_mnist() 함수에서도 (2 번째 이후의 읽기 시에) pickle을 이용한다.
  - pickle 덕분에 MNIST 데이터를 순식간에 준비할 수 있는 것이다.
---

- MNIST 이미지를 화면으로 불러보도록 하자. 

In [11]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image


def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

img = x_train[0]
label = t_train[0]
print(label)  # 5

print(img.shape)  # (784,)
img = img.reshape(28, 28)  # 형상을 원래 이미지의 크기로 변형
print(img.shape)  # (28, 28)

img_show(img)

5
(784,)
(28, 28)


- 주의
  - flatten=True로 설정해 읽어 들인 이미지는 1차원 넘파이 배열로 저장돼 있다. 그래서 이미지를 표시할 땐 원래 형상인 28 x 28 크기로 다시 변형해야 한다. reshape() 메서드에 원하는 형상을 인수로 지정하면 넘파이 배열의 형상을 바꿀 수 있다.
  - 또한 넘파이로 저장된 이미지 데이터를 PIL용 데이터 객체로 변환해야 하며, 이 변환은 Image.fromarray()가 수행한다.

---

- 신경망의 추론 처리
  - 드디어 이 MNIST 데이터셋을 가지고 추론을 수행하는 신경망을 구현해보자
  - 이 신경망은 입력층 뉴런을 784(28x28)개, 출력층 뉴련을 10개로 구성한다. 
  - 한편, 은닉층은 총 두 개로, 첫 번째 은닉층에는 50개의 뉴런을 두 번째 은닉층에는 100개의 뉴런을 배치한다. 여기서 50과 100은 임의로 정한 값이다. 

In [18]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 
import numpy as np
import pickle
from dataset.mnist import load_mnist
from common.functions import sigmoid, softmax

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test


def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network


def predict(network, x):
    w1, w2, w3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, w1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, w2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, w3) + b3
    y = softmax(a3)

    return y


- init_network()에서는 pickle 파일인 sample_weight.pkl에 저장된 학습된 가중치 매개변수를 읽는다. 이 파일에는 가중치와 편향 매개변수가 딕셔너리 변수로 저장되어 있다.
- 이제 이 세 함수를 이용해 신경망에 의한 추론을 수행해보고, 정확도(분류가 얼마나 올바른가)도 평가해보자

In [19]:
x, t = get_data()
network = init_network()

batch_size = 100 # 배치 크기
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


- 또한 이 예에서는 load_mnist 함수의 인수인 normalize를 True로 설정했다. 이처럼 데이터를 특정 범위로 변환하는 처리를 정규화라 하고, 신경망의 입력 데이터에 특정 변환을 가하는 것을 전처리라 한다. 여기에서는 입력 이미지 데이터에 대한 전처리 작업으로 정규화를 수행한 셈이다.

---

- 배치처리 
  - 입력 데이터와 가중치 매개변수의 형상에 주의하여 조금 전 구현을 다시 살펴보자
  - 우선 앞서 구현한 신경망 각 층의 가중치 형상을 출력해보자

In [21]:
x,_ = get_data()
network = init_network()
W1,W2,W3 = network['W1'], network['W2'], network['W3']

print(x.shape)
print(x[0].shape)
print(W1.shape) #1
print(W2.shape) #2
print(W3.shape) #3

(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


- 이 결과에서 다차원 배열의 대응하는 차원의 수가 일치함을 확인할 수 있다. 
  - (교재 102p를 참고해보자)
- 이는 이미지 데이터 1장만 입력했을 때의 데이터 처리 흐름이다. 
  - 그렇다면 이미지 여러 장을 한 번에 입력하는 경우를 생각해보자
    - 가령 이미지 100개를 묶어 predict() 함수에 한 번에 넘기는 경우가 있다고 해보자. 
    - x의 형상을 100x784fh qkRNjtj 100장 분량의 데이터를 하나의 입력 데이터로 표현하면 될 것이다. 
    - 이때 입력 데이터의 형상은 100x784, 출력 데이터의 형상은 100x10이 된다.
    - 이는 100장 분량의 입력 데이터의 결과가 한 번에 출력됨을 나타낸다. 
    - 가령 x[0]과 y[0]에는 0번째 이미지와 그 추론 결과가 저장되는 식이다.
    - 이처럼 하나로 묶은 입력 데이터를 배치(batch)라고 한다. 즉, 묶음을 의미하며 이미지가 지폐처럼 다발로 묶여 있다고 생각하면 된다.

- 배치 처리를 구현해보자

In [23]:
x, t = get_data()
network = init_network()

batch_size = 100 # 배치 크기
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1) # 100x10이라는 다차원에서 0번째 차원말고 1번째 차원에 접근하겠다. #??????????????
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


----

In [27]:
#??????????????????의 부연설명
x = np.array([[3,2,1],[4,5,6]])
y = np.argmax(x,axis=1)
print(y)

[0 2]


---

- 마지막으로 배치 단위로 분류한 결과를 실제 답과 비교한다. 이를 위해 ==연사자를 사용한다.

In [29]:
y = np.array([1,2,1,0])
t = np.array([1,2,0,0])
print(y==t)
np.sum(y==t)

[ True  True False  True]


3

- 이런 식으로 구현하면 된다.

-----

> ### Conclusion
      
      이번 장에서는 신경망의 순전파를 살펴봤다. 이번 장에서 설명한 신경망은 각 층의 뉴런들이 다음 층의 뉴런으로 신호를 전달한다는 점에서 앞 장의 퍼셉트론과 같다. 하지만 다름 뉴런으로 갈 때, 신호를 변화시키는 활성화 함수에 큰 차이가 있었다. 신경망에서는 매끄럽게 변화하는 시그모이드 함수를, 퍼셉트론에서는 갑자기 변화하는 계단 함수를 활성화 함수로 사용했다. 이 차이가 신경망 학습에 중요하다. 다음 장에서 더욱 알아보도록 하자.